## Importing necessary packages

In [101]:
import datetime as dt

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

pd.options.mode.chained_assignment = None

## Reading data

In [102]:
# Read data
df_bank = pd.read_csv(
    "../data/no_duplicates_data.csv", index_col="date", parse_dates=True
)
accounts = pd.read_csv("../data/no_duplicates_accounts.csv")
transactions = pd.read_csv("../data/no_duplicates_transactions.csv")

In [103]:
df_bank

,id,balance,update_date,amount
date,,,,
2020-10-16,0,13.63,2021-07-02,200.00
2020-10-16,0,13.63,2021-07-02,-192.00
2020-10-16,0,13.63,2021-07-02,200.00
2020-10-16,0,13.63,2021-07-02,-24.00
2020-10-16,0,13.63,2021-07-02,-50.00
...,...,...,...,...
2021-06-30,1252,2074.71,2021-07-02,-80.10
2021-06-30,1252,2074.71,2021-07-02,-46.05
2021-06-30,1252,2074.71,2021-07-02,-10.00


In [104]:
transactions

,account_id,date,amount
0,0,2020-10-16,200.00
1,0,2020-10-16,-192.00
2,0,2020-10-16,200.00
3,0,2020-10-16,-24.00
4,0,2020-10-16,-50.00
...,...,...,...
405477,1252,2021-06-30,-80.10
405478,1252,2021-06-30,-46.05
405479,1252,2021-06-30,-10.00
405480,1252,2021-07-01,-34.80


In [105]:
accounts

,id,balance,update_date
0,0,13.63,2021-07-02
1,1,12.91,2021-07-02
2,4,2806.75,2021-07-02
3,7,0.38,2021-07-02
4,8,25.04,2021-07-02
...,...,...,...
715,1247,688.03,2021-07-02
716,1249,5117.09,2021-07-02
717,1250,-2952.06,2021-07-02
718,1251,361.86,2021-07-02


## Define functions of preprocessing

In [106]:
def positive_transactions(x):
    """
    Returns only positive transactions, 
    and set 0 to negative ones.
    """
    if x >= 0:
        return x
    else:
        return 0

In [107]:
def negative_transactions(x):
    """
    Returns only negative transactions, 
    and set 0 to positive ones.
    """

    if x <= 0:
        return x
    else:
        return 0

In [108]:
def preprocess(df_kept_accounts, accounts_kept, nb_month=6):

    """
    This function returns preprocessed data that will be used as input 
    to the ML models
    """

    list_of_df = []

    for id in set(accounts_kept["id"]):

        print(id)

        # Transactions datframe df_id related to id
        df_id = df_kept_accounts[df_kept_accounts["id"] == id]

        # Create new columns "positive_transactions" and "negative_transactions"
        df_id["positive_transactions"] = df_id["amount"].apply(
            lambda x: positive_transactions(x)
        )
        df_id["negative_transactions"] = df_id["amount"].apply(
            lambda x: negative_transactions(x)
        )

        # Create "balance_per_day" column of balances for each date
        df_id = df_id.groupby("date").sum()
        df_id.sort_values(by="date", ascending=False, inplace=True)
        df_id["balance_per_day"] = -df_id["amount"]
        df_id["balance_per_day"].iloc[0] += accounts_kept[
            accounts_kept["id"] == id
        ].balance.values[0]
        df_id["balance_per_day"] = df_id["balance_per_day"].cumsum()

        df_id = df_id.drop(["balance", "id"], axis=1)

        df_id.sort_values(by="date", ascending=True, inplace=True)

        # Normalize data so that I solve sparcity of dates (add missing dates in df_id)
        for i, date in enumerate(df_id.index[:-1]):

            days = (df_id.index[i + 1] - df_id.index[i]).days
            if days == 1:
                continue

            else:
                for j in range(1, days):
                    df_id.loc[dt.timedelta(days=j) + date] = df_id.loc[date]
                    df_id.loc[
                        dt.timedelta(days=j) + date,
                        ["amount", "positive_transactions", "negative_transactions"],
                    ] = [0, 0, 0]

        df_id.sort_values(by="date", ascending=True, inplace=True)

        df_id["sum_outgoing_next_30_days"] = None

        # For each day of df_id compute sum of outgoings and incomes for next month
        for i, date in enumerate(df_id.index[:-30]):
            df_id.loc[date, "sum_outgoing_next_30_days"] = df_id.iloc[i + 1 : i + 31][
                "negative_transactions"
            ].sum()
            df_id.loc[date, "sum_income_next_30_days"] = df_id.iloc[i + 1 : i + 31][
                "positive_transactions"
            ].sum()

        for i in range(nb_month):
            df_id["negative_transaction_last_" + str(i + 1) + "_month"] = df_id[
                "sum_outgoing_next_30_days"
            ].shift((i + 1) * 30)
            df_id["positive_transaction_last_" + str(i + 1) + "_month"] = df_id[
                "sum_income_next_30_days"
            ].shift((i + 1) * 30)

        df_id = df_id.dropna()
        length = len(df_id)
        df_id = df_id.reset_index()

        # Monthly resampling of df_id
        df_id = df_id.drop([i for i in range(length) if i % 30 != 0])

        list_of_df.append(df_id)

    return list_of_df

## Preprocessing data 

In [109]:
# Call function preprocess
list_of_df = preprocess(df_bank, accounts)

0
1
4
7
8
10
12
15
16
17
18
19
23
25
26
27
28
29
31
35
37
38
39
40
41
45
46
47
48
49
50
51
53
54
55
57
58
59
61
62
64
65
68
74
80
81
83
84
85
90
91
92
93
94
96
97
99
100
101
106
107
109
110
111
112
116
118
123
124
126
127
128
129
131
133
134
141
143
144
145
147
148
149
150
152
155
156
157
158
161
162
163
164
165
166
169
172
173
175
177
178
179
180
181
182
184
185
186
187
189
190
192
196
199
200
202
203
210
212
214
217
218
221
222
223
224
225
227
228
230
231
232
233
235
236
240
243
244
245
251
253
254
256
257
258
261
263
265
267
269
270
274
275
276
278
279
283
284
285
286
287
288
289
292
293
296
300
301
302
304
305
308
310
311
312
315
316
318
319
321
322
323
325
326
327
328
331
335
338
339
340
343
348
352
355
358
359
360
362
363
369
375
378
380
381
383
385
387
388
390
391
392
393
394
395
396
397
398
399
400
401
403
404
405
408
410
413
414
415
418
420
424
425
426
427
431
432
434
436
440
442
444
445
446
447
448
451
452
454
457
458
459
461
462
466
471
472
475
477
478
479
480
481
484
485
48

In [110]:
# Convert list of datas to one Dataframe
df_final = pd.DataFrame()
for i, d in enumerate(list_of_df):
    print(i)
    df_final = pd.concat([df_final, d], axis=0)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [111]:
df_final

,date,amount,positive_transactions,negative_transactions,balance_per_day,sum_outgoing_next_30_days,sum_income_next_30_days,negative_transaction_last_1_month,positive_transaction_last_1_month,negative_transaction_last_2_month,positive_transaction_last_2_month,negative_transaction_last_3_month,positive_transaction_last_3_month,negative_transaction_last_4_month,positive_transaction_last_4_month,negative_transaction_last_5_month,positive_transaction_last_5_month,negative_transaction_last_6_month,positive_transaction_last_6_month
0,2021-04-14,0.00,0.00,0.00,18.36,-3225.94,3215.50,-1585.2,1587.00,-1000.0,1000.00,-1070.29,1066.00,-3562.99,1828.00,-102.0,1841.00,-5037.0,1952.00
30,2021-05-14,0.00,0.00,0.00,7.92,-5186.1,5185.80,-3225.94,3215.50,-1585.2,1587.00,-1000.0,1000.00,-1070.29,1066.00,-3562.99,1828.00,-102.0,1841.00
0,2021-05-08,0.00,165.00,-165.00,4.41,-835.0,831.00,-7165.81,7165.00,-556.54,205.00,-9818.04,10252.33,-4768.09,4770.40,-5929.04,5896.49,-747.32,770.00
0,2021-03-10,-316.98,532.92,-849.90,1962.71,-11866.82,14875.79,-15941.46,16689.22,-12972.15,12515.82,-9490.21,8749.92,-15066.06,9488.18,-11310.98,10098.09,-4196.96,12120.28
30,2021-04-09,820.83,1318.93,-498.10,3833.87,-11306.21,8283.15,-11866.82,14875.79,-15941.46,16689.22,-12972.15,12515.82,-9490.21,8749.92,-15066.06,9488.18,-11310.98,10098.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,2021-01-07,-632.59,189.90,-822.49,1737.22,-6727.94,6617.65,-8465.18,8279.85,-3363.25,4500.00,-852.69,1005.72,0.0,0.00,0.0,0.00,0.0,0.00
270,2021-02-06,-1125.61,0.00,-1125.61,2119.95,-5271.7,7420.93,-6727.94,6617.65,-8465.18,8279.85,-3363.25,4500.00,-852.69,1005.72,0.0,0.00,0.0,0.00
300,2021-03-08,-401.63,0.00,-401.63,3545.20,-6758.57,6358.83,-5271.7,7420.93,-6727.94,6617.65,-8465.18,8279.85,-3363.25,4500.00,-852.69,1005.72,0.0,0.00
330,2021-04-07,-142.22,0.00,-142.22,2886.05,-5145.89,3000.00,-6758.57,6358.83,-5271.7,7420.93,-6727.94,6617.65,-8465.18,8279.85,-3363.25,4500.00,-852.69,1005.72


## Save data

In [112]:
# Save dataframe
df_final.to_csv("../data/df_final_monthly.csv", index=True)